In [1]:
import os
import glob

import numpy as np
import nibabel as nib
from tqdm import tqdm

In [2]:
def binarize_segmentation(img_np, only_specific_vessel_class: int = None):
    if not only_specific_vessel_class:
        img_np = np.where(img_np >= 1, 1, 0)
    else:
        img_np = np.where(img_np == only_specific_vessel_class, 1, 0)
    return img_np


In [3]:
def transform_label_vols_in_dir(filepaths: list, output_dir: str, transformation: callable):
    os.makedirs(output_dir, exist_ok=True)
    for file_path in tqdm(filepaths):
        img_name = os.path.basename(file_path)
        img = nib.load(file_path)
        img_np = img.get_fdata()
        img_np = transformation(img_np)
        img = nib.Nifti1Image(img_np, img.affine, img.header)
        nib.save(img, os.path.join(output_dir, img_name))

## Remap label data

In [4]:
data_dir = os.path.join('..', '..', '..', 'data', 'ADAM', 'Dataset005_ADAM3ClassesAneurysmsOnly')
data_dst_dir = os.path.join('..', '..', '..', 'data', 'ADAM', 'Dataset005_ADAMBinaryAneurysmsOnly')
os.makedirs(data_dst_dir, exist_ok=True)

labels_tr_dir_org = os.path.join(data_dir, 'labelsTr', '*.nii.gz')
labels_tr_dst_org = os.path.join(data_dst_dir, 'labelsTr')

labels_ts_dir_org = os.path.join(data_dir, 'labelsTs', '*.nii.gz')
labels_ts_dst_dir = os.path.join(data_dst_dir, 'labelsTs')

file_paths_train = glob.glob(labels_tr_dir_org)
file_paths_test = glob.glob(labels_ts_dir_org)

In [6]:
# Train
transform_label_vols_in_dir(
    file_paths_train, 
    labels_tr_dst_org,
    binarize_segmentation
)

# Test
transform_label_vols_in_dir(
    file_paths_test, 
    labels_ts_dst_dir,
    binarize_segmentation
)

100%|██████████| 23/23 [03:01<00:00,  7.89s/it]


In [7]:
train_dir_org = os.path.join(data_dir, 'imagesTr')
train_dir_dst = os.path.join(data_dst_dir, 'imagesTr')

In [8]:
!ln -s $train_dir_org $train_dir_dst

In [9]:
test_dir_org = os.path.join(data_dir, 'imagesTs')
test_dir_dst = os.path.join(data_dst_dir, 'imagesTs')

In [10]:
!ln -s $test_dir_org $test_dir_dst

## Map to only untreated aneurysm

In [6]:
data_dst_dir = os.path.join('..', '..', '..', 'data', 'ADAM', 'Dataset007_ADAMBinaryUntreatedAneurysmsOnly')
os.makedirs(data_dst_dir, exist_ok=True)

labels_tr_dir_org = os.path.join(data_dir, 'labelsTr', '*.nii.gz')
labels_tr_dst_org = os.path.join(data_dst_dir, 'labelsTr')

labels_ts_dir_org = os.path.join(data_dir, 'labelsTs', '*.nii.gz')
labels_ts_dst_dir = os.path.join(data_dst_dir, 'labelsTs')

file_paths_train = glob.glob(labels_tr_dir_org)
file_paths_test = glob.glob(labels_ts_dir_org)

# Train
transform_label_vols_in_dir(
    file_paths_train, 
    labels_tr_dst_org,
    lambda vol: binarize_segmentation(vol, only_specific_vessel_class=1)
)
# Test
transform_label_vols_in_dir(
    file_paths_test, 
    labels_ts_dst_dir,
    lambda vol: binarize_segmentation(vol, only_specific_vessel_class=1)
)

100%|██████████| 23/23 [00:18<00:00,  1.23it/s]


In [12]:
train_dir_org = os.path.join(data_dir, 'imagesTr')
train_dir_dst = os.path.join(data_dst_dir, 'imagesTr')

In [13]:
!ln -s $train_dir_org $train_dir_dst

In [14]:
test_dir_org = os.path.join(data_dir, 'imagesTs')
test_dir_dst = os.path.join(data_dst_dir, 'imagesTs')

In [15]:
!ln -s $test_dir_org $test_dir_dst